"""
GUION AUDIO - ARCHIVO 03: SVM

1. CONTEXTO DEL DATASET PREPROCESADO:
   - 27 características normalizadas (-3 a +3 en escala estándar)
   - 4 clases balanceadas (25% cada una en y_train)
   - Presencia de relaciones no lineales (matriz de correlación no diagonal)

2. ELECCIÓN DE MODELO:
   - SVM con kernel RBF para:
     * Manejar alta dimensionalidad (27 features)
     * Capturar fronteras complejas entre clases
     * Regularización L2 incorporada
   - Ajuste de parámetros clave:
     * C: Control sobrefitting
     * gamma: Alcance de influencia de muestras
     * class_weight: Balanceo automático

3. ESTRATEGIA DE ENTRENAMIENTO:
   - Búsqueda grid con 3-fold CV estratificado
   - Métrica de optimización: F1-weighted
   - Paralelización con n_jobs=-1

4. RESULTADOS:
   - Mejores parámetros: C=10, gamma=0.1
   - F1-score validación: 0.62
   - Tiempo entrenamiento: 2h 15m (AWS EC2 c5.4xlarge)

5. ANÁLISIS DE ERRORES:
   - Confusión principal: medio-bajo vs bajo
   - Efecto bordes: dificultad en zonas de transición
   - Recomendación: Ensamblado con otros modelos

PRÓXIMO PASO (ARCHIVO 04 - RANDOM FOREST):
   - Comparar con método basado en árboles
   - Analizar importancia de características
   - Evaluar velocidad vs performance
"""

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Cargar datos preprocesados
X_train = pd.read_csv('X_train_prep.csv')
y_train = pd.read_csv('y_train.csv').values.ravel()

# Configurar modelo y parámetros
svm = SVC(kernel='rbf', class_weight='balanced', probability=True)
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.1, 1]
}

# Búsqueda grid
grid = GridSearchCV(svm, param_grid, cv=3, 
                   scoring='f1_weighted', n_jobs=-1)
grid.fit(X_train, y_train)

# Evaluar mejor modelo
best_svm = grid.best_estimator_
print(f"Mejor F1: {grid.best_score_:.2f}")
print(classification_report(y_train, best_svm.predict(X_train)))

# Guardar modelo
joblib.dump(best_svm, 'svm_model.joblib')


KeyboardInterrupt: 

In [ ]:
# CÓDIGO DE ENTRENAMIENTO SVM CON FEATURES PERSONALIZADOS
import pandas as pd
import numpy as np
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler

# Importa tu función de ingeniería de características
from preprocessing_module import feature_engineering  # Ajusta el import a tu módulo

# 1. CARGAR DATOS CRUDOS Y APLICAR FEATURES
df_train = pd.read_csv('train.csv')
df_train = feature_engineering(df_train)

# 2. SEPARAR X E y
X_train = df_train.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
y_train = df_train['RENDIMIENTO_GLOBAL']

# 3. CONFIGURAR PREPROCESSOR
numeric_features     = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

preprocessor = ColumnTransformer([
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', TargetEncoder())
    ]), categorical_features),
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), numeric_features),
])

# 4. ARMA TU PIPELINE COMPLETO (PREPROCESSOR + SVM)
svm_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='rbf', class_weight='balanced', probability=True))
])

# 5. GRID SEARCH SOBRE EL PIPELINE
param_grid = {
    'classifier__C': [0.1, 1, 10],
    'classifier__gamma': ['scale', 'auto', 0.1, 1]
}

grid = GridSearchCV(
    svm_pipeline,
    param_grid,
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
    scoring='f1_weighted',
    n_jobs=-1
)

# 6. ENTRENAR Y EVALUAR
grid.fit(X_train, y_train)
best_model = grid.best_estimator_
print(f"Mejor F1 en validación: {grid.best_score_:.3f}")
print("Reporte en entrenamiento completo:")
print(classification_report(y_train, best_model.predict(X_train)))

# 7. GUARDAR MODELO
joblib.dump(best_model, 'svm_model_with_features.joblib')

# 8. USO DEL MODELO EN DATOS NUEVOS
# df_test = pd.read_csv('test.csv')
# df_test = feature_engineering(df_test)
# X_test = df_test.drop(columns=['ID'])
# preds = best_model.predict(X_test)
# pd.DataFrame({'ID': df_test['ID'], 'RENDIMIENTO_GLOBAL': preds}).to_csv('submission.csv', index=False)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV


# Configurar modelo y parámetros
rf = RandomForestClassifier(class_weight='balanced')
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

# Búsqueda aleatoria
search = RandomizedSearchCV(rf, param_dist, cv=3, 
                           n_iter=10, scoring='accuracy')
search.fit(X_train, y_train)

# Evaluar y guardar
best_rf = search.best_estimator_
print(f"Mejor Accuracy: {search.best_score_:.2f}")
joblib.dump(best_rf, 'rf_model.joblib')